# Como saber si mi caja negra esta bien entrenada?

Haciendo un repaso:

1. Obtuvimos datos utiles para resolver nuestro problema y los convertimos en una tabla
2. Separamos nuestra tabla en dos: una tabla de **Entrenamiento** y una de **Prueba**
3. Utilizamos la tabla de entrenamiento para Entrenar una caja negra que me permite hacer predicciones

**Ahora utilizaremos la tabla de Prueba** para saber si nuestra caja negra esta bien entrenada aprovechando que en esta tabla de Prueba tenemos **datos que la caja negra nunca ha visto** y de los cuales **sabemos cuales deberian ser las predicciones correctas**.

![Error de prediccion de caja negra](img/error_prediccion_caja_negra.png)

Como dijimos ya hay metricas definidas y bien estudiadas para poder medir que tan bien entrenada esta una caja negra. Utilizaremos distintas medidas segun sean de **Regresion** o de **Clasificacion** 

Ahora solo veremos una para cada metodo y mas adelante en esta unidad veremos mas.

**Para regresion:**
- **Error cuadratico medio**: es la suma de las diferencias elevadas al cuadrado. Las diferencias se refieren a la resta entre cada valor predecido y el real. Cuanto menor sea este error mejor entrenada estara nuestra caja negra de regresion.

**Para clasificacion:**
- **Porcentaje de aciertos (Accuracy)**: cuantas predicciones hemos acertado dividido el total de predicciones. Cuanto mayor sea la cantidad de aciertos mejor entrenada estara nuestra caja negra de clasificacion.

In [ ]:
from sklearn import metrics

In [3]:
# supongamos que los datos verdaderos son
y_verdadero = [1.0, 2.0, 3.0, 4.0]
y_predecido = [1.3, 2.1, 3.2, 4.8]

In [4]:
metrics.mean_squared_error(y_verdadero, y_predecido)

0.009999999999999995

In [5]:
# supongamos que los datos verdaderos son
y_verdadero = ["benigno", "maligno", "maligno", "benigno", "benigno"]
y_predecido = ["benigno", "maligno", "maligno", "maligno", "benigno"]

In [7]:
metrics.accuracy_score(y_verdadero, y_predecido)

0.8

## Sesgo y Varianza (Bias y Variance)

Aqui veremos cuales son los problemas mas comunes al entrenar una caja negra. Como detectarlos y como corregirlos. Para ello empezaremos calculando el error para las dos tablas (Entrenamiento y Prueba) por separado. 

![Error de prediccion de caja negra](img/train_test_errors.png)

Una vez calculados los errores para ambas tablas analizaremos los cuatro posibles casos donde el error de entrenamiento puede ser bajo o alto y para cada uno de estos casos el error de prueba puede ser similar al de entrenamiento o mucho mayor:

**Sesgo Bajo y Varianza Alta:**
- Error de Entrenamiento:  1% (**bajo**)
- Error de Prueba: 10% (**mucho mayor al de entrenamiento**)

**Sesgo Alto y Varianza Baja:**
- Error de Entrenamiento: 15% (**alto**)
- Error de Prueba: 16% (**similar al de entrenamiento**)

**Sesgo Alto y Varianza Alta:**
- Error de Entrenamiento: 15% (**alto**)
- Error de Prueba: 30% (**mucho mayor al de entrenamiento**)

**Sesgo Bajo y Varianza Baja (lo que buscamos):**
- Error de Entrenamiento:  0.5% (**bajo**)
- Error de Prueba:  1.0% (**bajo**)

Como vemos el objetivo es obtener los errores de Entrenamiento y Prueba bajos.

- El **Error de Entrenamiento alto** (tambien conocido como **Underfitting** en ingles) puede estar dado por una de las siguientes condiciones:
  - Necesitamos mas datos para entrenar nuestra caja negra
  - Necesitamos una caja negra que predecir datos complejos de predecir, por ejemplo los ensambles pueden predecir datos mas complejos que una regresion lineal o naive bayes porque no asumen ninguna hipotesis respecto a los datos.
- El **Error de Prueba alto** (tambien conocido como **Overfitting** en ingles) puede estar dado por una de las siguientes condiciones:
  - Necesitamos mas datos para entrenar nuestra caja negra
  - Necesitamos una caja negra mas simple o un metodo para evitar que la caja negra se ajuste demasiado a los datos de prueba. Estos metodos se llaman de **regularizacion**

## Otras metricas

**Para clasificacion:**
    
Cuando realizamos una clasificacion intervienen varios factores:
1. Puede haber muchos mas datos de una clase que de otra y nos llevara a un accuracy alto pero que no es util
2. La mayoria de los algoritmos de clasificacion nos da la probabilidad de que un dato sea de una clase u otra por lo que deberemos elegir un limite a partir del cual decimos que es de una clase u otra.
    
- Accuracy.
- Precision: TP / (TP + FP)
- Recall: TP / (TP + FN)
- F1 Score: 2 / (1/P + 1/R)
- ROC curve: the precision and recall measures for different sigmoid thresholds
- Area Under de Curve: is the measure of the ability of a classifier to distinguish between classes and is used as a summary of the ROC curve

In [ ]:
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import roc_curve, auc

y_verdadero = ["benigno", "maligno", "maligno", "benigno", "benigno"]
y_prob = [0.2, 0.91, 0.85, 0.55, 0.3]
# suponiendo que elegimos como unbral 0.5
y_predecido = ["benigno", "maligno", "maligno", "maligno", "benigno"]

metrics.precision_score(y_verdadero, y_predecido)
metrics.recall_score(y_verdadero, y_predecido)

fpr, tpr, thresholds = metrics.roc_curve(y_verdadero, y_prob, pos_label="malo")
metrics.auc(fpr, tpr)

**Para regresion:**

Ademas del error cuadratico medio otra metrica muy utilizada es R2 que permite obtener un valor mas ameno ya que varia entre 0 y 1.

In [ ]:
metrics.r2_score(y_verdadero, y_predecido)

# Fin: [Volver al contenido del curso](https://www.freecodingtour.com/cursos/espanol/datascience/datascience.html)